In [1]:
from Functions import LoadData
import datetime
import pandas as pd
pd.options.plotting.backend = "plotly"
import os
import numpy as np
import plotly.express as px

In [2]:
df, df_fine_patient_phases, medication_settings, medTimes = LoadData.get_data('EM2',path='C:/data/raw/EMERGENT/')
df.index=df['time']

C:\workspace\EMERGENT\Functions\LoadData.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fine_patient_phases['end_date'].iloc[:-1]=df_fine_patient_phases['start_date'].iloc[1:]
C:\workspace\EMERGENT\Functions\LoadData.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fine_patient_phases['end_date'].iloc[-1]=df_combined['time'].iloc[-1]


In [59]:
# Average over med intakes
param = 'probability_tremor'
# Smooth first
smoothed = df[param].groupby(pd.Grouper(freq='10Min')).aggregate(np.mean)

t = medTimes['date'].iloc[0]
td = datetime.timedelta(hours=1)
tc = smoothed[t-td:t+td].values
timeseries = np.zeros((medTimes.shape[0],tc.shape[0]))
for r in range(medTimes.shape[0]):
    t = medTimes['date'].iloc[r]
    tc = smoothed[t-td:t+td].values
    timeseries[r,:]=tc
    
timeseries[np.isnan(timeseries)]=0
#px.imshow(timeseries,aspect='auto')
px.line(x=np.arange(tc.shape[0]),y=np.nanmean(timeseries,axis=0))

In [52]:
# Average over programmings

# Score
param = 'probability_dyskinesia'
smoothed = df[param].groupby(pd.Grouper(freq='15Min')).aggregate(np.mean)


#Get days for the adjustments
changes=np.argwhere(np.any(np.diff(df_fine_patient_phases.iloc[:,2:],axis=0)!=0,axis=1))+1
changes = [c[0] for c in changes]

adjustments = df_fine_patient_phases['start_date'][changes]
t = adjustments.iloc[0]
td = datetime.timedelta(days=2)
tc = smoothed[t-td:t+td].values
timeseries = np.zeros((adjustments.shape[0],tc.shape[0]))

for r in range(adjustments.shape[0]):
    t = adjustments.iloc[r]
    tc = smoothed[t-td:t+td].values
    timeseries[r,:]=tc
    
timeseries[np.isnan(timeseries)]=0
#print(np.sum(timeseries==np.nan))
px.imshow(timeseries)
px.line(x=np.arange(tc.shape[0]),y=np.nanmean(timeseries,axis=0))


In [24]:
#Average over days
# Score
param = 'probability_dyskinesia'
smoothed = df[param].groupby(pd.Grouper(freq='15Min')).aggregate(np.mean)

for phase in range(df_fine_patient_phases.shape[0]):
    days=[]
    start = df_fine_patient_phases['start_date'].iloc[phase]
    end = df_fine_patient_phases['end_date'].iloc[phase]
    diffDays=(end-start).days
    print(diffDays)
    td = datetime.timedelta(days=1)
    for i in range(diffDays):
        d = smoothed[np.isin(smoothed.index.date,start+i*td)].values
        days.append(d)
print(len(days))

44
15
7
7
2
11
14
7
7
15
6
6
